### New Email Analysis for my gmail data collected on the 16th of March, 2024.

In [ ]:
# Import libraries 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import mailbox

In [ ]:
# Put mbox file into a variable
mboxfile = 'All mail Including Spam and Trash.mbox'
mbox = mailbox.mbox(mboxfile)
mbox 

In [ ]:
type(mbox)

In [ ]:
print(mbox)

In [ ]:
# List of available keys in the mbox file
for key in mbox[0].keys():
    print(key)

#### Data Transformation

In [ ]:
# Data Cleaning
import csv 

In [ ]:
# Create a CSV file with only the required attributes:
with open('mailbox.csv', 'w') as outputfile:
    writer = csv.writer(outputfile)
    writer.writerow(['subject', 'from', 'date', 'to', 'label', 'thread'])
    for message in mbox:
        writer.writerow([
            message['subject'], message['from'], message['date'],
            message['to'], message['X-Gmail-Labels'], message['X-GM-THRID']
        ])

In [ ]:
# Load the csv file
dfs = pd.read_csv('mailbox.csv')

In [ ]:
dfs.head()

In [ ]:
# Summary info
dfs.info()

In [ ]:
# Convert date feature to a datetime format
dfs['date'] = dfs['date'].apply(lambda x: pd.to_datetime(x,
errors='coerce', utc=True))

In [ ]:
dfs.info()

In [ ]:
# Checking for missing values in the date column
dfs['date'].isnull().sum()

In [ ]:
missingData = dfs[dfs['date'].isnull()]
missingData.head()

In [ ]:
# Removing NaN
dfs = dfs[dfs['date'].notna()]

In [ ]:
dfs['date'].isnull().sum()

In [ ]:
# Saving to csv file
dfs.to_csv('gmail.csv')

In [ ]:
dfs.info()

In [ ]:
dfs.head(10)

#### Data refactoring

In [ ]:
# import regular expression
import re 

In [ ]:
# let's create a function that takes an entire string from any column and
# extracts an email address:
def extract_email_ID(string):
    email = re.findall(r'<(.+?)>', string)
    if not email:
        email = list(filter(lambda y: '@' in y, string.split()))
    return email[0] if email else np.nan

In [ ]:
dfs['from'] = dfs['from'].apply(lambda x: extract_email_ID(x))

In [ ]:
dfs.head()

In [ ]:
myemail = ''
dfs['label'] = dfs['from'].apply(lambda x: 'sent' if x==myemail
else 'inbox')

In [ ]:
dfs.head()

In [ ]:
# Drop the to column
dfs.drop(columns='to', inplace=True)

In [ ]:
dfs.head()

In [ ]:
# Refactor the date column
import datetime
import pytz

In [ ]:
# List of all time zones
#pytz.all_timezones

In [ ]:
def refactor_timezone(x):
    est = pytz.timezone('Africa/Lagos')
    return x.astimezone(est)

In [ ]:
dfs['date'] = dfs['date'].apply(lambda x: refactor_timezone(x))

In [ ]:
dfs.head()

In [ ]:
# Convert the day of the week into names of the day of the week
dfs['dayofweek'] = dfs['date'].apply(lambda x: x.day_name())

In [ ]:
dfs.head()

In [ ]:
# Convert to dayofweek to category
dfs['dayofweek'] = dfs.dayofweek.astype('category')

In [ ]:
dfs.info()

In [ ]:
# Refactor for time of day
dfs['timeofday'] = dfs['date'].apply(lambda x: x.hour + x.minute/60 + x.second/3600)

In [ ]:
# Refactor for hour
dfs['hour'] = dfs['date'].apply(lambda x: x.hour)

In [ ]:
# Refactor for year integer
dfs['year_int'] = dfs['date'].apply(lambda x: x.year)

In [ ]:
# Refactor for year fraction
dfs['year'] = dfs['date'].apply(lambda x: x.year + x.dayofyear/365.25)

In [ ]:
dfs.head()

In [ ]:
dfs.to_csv('gmail_data.csv')

In [ ]:
# Set date to index
dfs.index = dfs['date']

In [ ]:
# Delete the date column because it's no longer relevant
del dfs['date']

In [ ]:
dfs.head()